In [2]:
Pkg.add("Convex")
Pkg.add("ECOS")
using DataFrames, PyPlot
using Convex
using PyPlot
using ECOS
set_default_solver(ECOSSolver(verbose=0))

INFO: Cloning cache of Convex from https://github.com/JuliaOpt/Convex.jl.git
INFO: Cloning cache of MathProgBase from https://github.com/JuliaOpt/MathProgBase.jl.git
INFO: Installing Convex v0.4.0
INFO: Installing MathProgBase v0.5.7
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of Convex
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Cloning cache of ECOS from https://github.com/JuliaOpt/ECOS.jl.git
INFO: Installing ECOS v0.6.7
INFO: Building ECOS
INFO: Attempting to Create directory C:\Users\admin\.julia\v0.5\ECOS\deps\downloads
INFO: Downloading file https://cache.julialang.org/https://bintray.com/artifact/download/tkelman/generic/ecos-2.0.2.7z
INFO: Done downloading file https://cache.julialang.org/https://bintray.com/artifact/download/tkelman/generic/ecos-2.0.2.7z



7-Zip [64] 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18

Processing archive: C:\Users\admin\.julia\v0.5\ECOS\deps\downloads\ecos-2.0.2.7z

Extracting  usr\COPYING


INFO: Attempting to Create directory C:\Users\admin\.julia\v0.5\ECOS
INFO: Directory C:\Users\admin\.julia\v0.5\ECOS already created


Extracting  usr\bin32\libecos.dll
Extracting  usr\bin64\libecos.dll
Extracting  usr\bin64
Extracting  usr\bin32
Extracting  usr

Everything is Ok

Folders: 3
Files: 3
Size:       718676
Compressed: 195537


INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of ECOS
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Recompiling stale cache file C:\Users\admin\.julia\lib\v0.5\DataStructures.ji for module DataStructures.
INFO: Precompiling module MathProgBase.
INFO: Precompiling module ECOS.

ECOS.ECOSSolver(Any[(:verbose,0)])

norm(Convex.AbstractExpr, Symbol) in module Convex at C:\Users\admin\.julia\v0.5\Convex\src\atoms/second_order_cone/norm.jl:45 overwritten at deprecated.jl:49.


In [3]:
sheet = readtable("2009_manhattan.csv");

In [4]:
size(sheet)

(18674,20)

In [5]:
reg_price=sheet[:SALE_PRICE].>1000
sheet=sheet[reg_price,:];
size(sheet)

(13918,20)

In [6]:
reg_sqft=sheet[:GROSS_SQUARE_FEET].>0
sheet=sheet[reg_sqft,:];
size(sheet)

(8486,20)

In [7]:
areas=unique(sheet[:NEIGHBORHOOD])

37-element DataArrays.DataArray{String,1}:
 "ALPHABET CITY"            
 "CHELSEA"                  
 "CHINATOWN"                
 "CIVIC CENTER"             
 "CLINTON"                  
 "EAST VILLAGE"             
 "FASHION"                  
 "FINANCIAL"                
 "FLATIRON"                 
 "GRAMERCY"                 
 "GREENWICH VILLAGE-CENTRAL"
 "GREENWICH VILLAGE-WEST"   
 "HARLEM-CENTRAL"           
 ⋮                          
 "MURRAY HILL"              
 "SOHO"                     
 "SOUTHBRIDGE"              
 "TRIBECA"                  
 "UPPER EAST SIDE (59-79)"  
 "UPPER EAST SIDE (79-96)"  
 "UPPER EAST SIDE (96-110)" 
 "UPPER WEST SIDE (59-79)"  
 "UPPER WEST SIDE (79-96)"  
 "UPPER WEST SIDE (96-116)" 
 "WASHINGTON HEIGHTS LOWER" 
 "WASHINGTON HEIGHTS UPPER" 

In [8]:
zips=unique(sheet[:ZIP_CODE])

45-element DataArrays.DataArray{Int64,1}:
 10009
 10002
 10011
 10001
 10010
 10013
 10038
 10007
 10036
 10019
 10003
 10018
 10005
     ⋮
 10022
 10065
 10021
 10075
 10044
 10028
 10128
 10023
 10024
 10069
 10033
 10040

## Build neigbourhood to avg price dictionary

In [9]:
area_avg=Dict([])

Dict{Any,Any} with 0 entries

In [10]:
for i=1:length(areas)
    current_area=areas[i]
    this_area=sheet[:NEIGHBORHOOD].==current_area
    area_prices=sheet[this_area,:SALE_PRICE];
    area_avg[current_area]=mean(area_prices)
end

In [11]:
area_avg["SOHO"]

3.012668663265306e6

# This is the X matrix !!

In [12]:
n=length(sheet[1])
d=8
X=zeros(n,d)
X=[X ones(n)] #add an offset

for i=1:n
    X[i,1]=area_avg[sheet[i,:NEIGHBORHOOD]]
    X[i,2]=sheet[i,:YEAR_BUILT]
    tax=sheet[i,:TAX_CLASS_AT_TIME_OF_SALE]
    X[i,2+tax]=1
    X[i,7]=sheet[i,:LAND_SQUARE_FEET]
    X[i,8]=sheet[i,:GROSS_SQUARE_FEET]
end

X

8486×9 Array{Float64,2}:
 1.37516e6  1920.0  0.0  1.0  0.0  0.0   2401.0   6920.0  1.0
 1.37516e6  1900.0  0.0  1.0  0.0  0.0   2212.0   9660.0  1.0
 1.37516e6  1920.0  0.0  1.0  0.0  0.0   4918.0  15392.0  1.0
 1.37516e6     0.0  0.0  1.0  0.0  0.0      0.0   1300.0  1.0
 1.37516e6     0.0  0.0  1.0  0.0  0.0      0.0   1135.0  1.0
 1.37516e6     0.0  0.0  1.0  0.0  0.0      0.0   1048.0  1.0
 1.37516e6     0.0  0.0  1.0  0.0  0.0      0.0   1054.0  1.0
 1.37516e6  1928.0  0.0  1.0  0.0  0.0   6442.0    450.0  1.0
 1.37516e6  1928.0  0.0  1.0  0.0  0.0   6442.0    490.0  1.0
 1.37516e6     0.0  0.0  1.0  0.0  0.0      0.0    740.0  1.0
 1.37516e6     0.0  0.0  1.0  0.0  0.0      0.0    750.0  1.0
 1.37516e6     0.0  0.0  1.0  0.0  0.0      0.0   1000.0  1.0
 1.37516e6     0.0  0.0  1.0  0.0  0.0      0.0    632.0  1.0
 ⋮                                 ⋮                         
 1.8725e6   1987.0  0.0  1.0  0.0  0.0      0.0    702.0  1.0
 1.8725e6   1987.0  0.0  1.0  0.0  0.0      0

In [13]:
y=convert(Array,sheet[:,:SALE_PRICE])

8486-element Array{Int64,1}:
  670509
 3500000
 1252452
 1299000
 1050000
  775000
  976000
  585000
  585000
  750000
  712500
  790000
  700000
       ⋮
   73164
   68141
  445000
   95593
  102560
  105453
  100793
  700000
   85000
  515000
 2800000
 1025000

## Break X in half into train and test set, same as y

In [33]:
flips=2*rand(n)
train_rows=flips.>=1
test_rows=flips.<1

8486-element BitArray{1}:
  true
 false
 false
  true
  true
 false
 false
 false
  true
 false
  true
 false
 false
     ⋮
  true
  true
  true
  true
 false
 false
 false
  true
  true
  true
  true
 false

In [34]:
X_train=X[train_rows,:]
X_test=X[test_rows,:]
y_train=y[train_rows]
y_test=y[test_rows]

4277-element Array{Int64,1}:
   670509
  1299000
  1050000
   585000
   712500
  1500000
  2000000
  7260000
  3400000
  2424000
  3653000
  6500000
 34962084
        ⋮
  5975000
  6530552
  3675000
  1996144
    73164
    68141
   445000
    95593
   700000
    85000
   515000
  2800000

# 1. solve L2 least squares

In [59]:
w=X_train\y_train

9-element Array{Float64,1}:
     1.27946  
 -2870.96     
     5.725e6  
     2.43011e6
     0.0      
    -7.49754e6
    -0.72885  
   173.928    
     6.57579e5

In [60]:
println("mean error squares on train set: ",mean((y_train-X_train*w).^2))
println("mean error squares on test set: ", mean((y_test-X_test*w).^2))

mean error squares on train set: 8.51993913656622e13
mean error squares on test set: 5.719730420644885e14


In [51]:
ys=[y_train X_train*w]

4209×2 Array{Float64,2}:
      3.5e6           1.07085e6
      1.25245e6       2.00842e6
 775000.0             5.02943e6
 976000.0             5.03047e6
 585000.0            -6.14496e5
 750000.0             4.97586e6
 790000.0             5.02108e6
 700000.0             4.95707e6
      2.125e6         5.32642e6
      1.5773e6        5.11058e6
      5.85494e6       6.00213e6
 675250.0            -4.64631e5
      1.0785e6        4.9776e6 
      ⋮                        
 155649.0             4.4837e6 
 304200.0             4.46039e6
 850000.0             1.01352e6
      2.6e6           5.78398e6
      2.0125e6        3.73273e6
      2.0125e6        3.73273e6
      5.0e6           9.15595e6
      3.5e6           9.52398e6
 102560.0        -85992.1      
 105453.0        -85992.1      
 100793.0        -76252.1      
      1.025e6        -9.26262e6

# 2. ridge and lasso regression

In [52]:
function ridge(X,y; λ=1)
    d = size(X,2)
    w = Variable(d)
    p = minimize(sumsquares(X*w - y) + λ*sumsquares(w))
    solve!(p)
    return w.value
end


function lasso(X,y; λ=1)
    d = size(X,2)
    w = Variable(d)
    p = minimize(sumsquares(X*w - y) + λ*norm(w,1))
    solve!(p)
    return w.value
end

lasso (generic function with 1 method)

In [57]:
w_ridge=ridge(X_train,y_train)
println("ridge mean error squares on train set: ",mean((y_train-X_train*w_ridge).^2))
println("ridge mean error squares on test set: ", mean((y_test-X_test*w_ridge).^2))

ridge mean error squares on train set: 9.85575399529831e13
ridge mean error squares on test set: 6.235261835196436e14


In [58]:
w_lasso=lasso(X_train,y_train)
println("lasso mean error squares on train set: ",mean((y_train-X_train*w_lasso).^2))
println("lasso mean error squares on test set: ", mean((y_test-X_test*w_lasso).^2))

lasso mean error squares on train set: 8.982938842018011e13
lasso mean error squares on test set: 5.898291498380994e14
